# Local registration with ICP

    In the RGBD folder we have the first 400 images from one of the datasets from: (http://redwood-data.org/indoor_lidar_rgbd/download.html)


    If you want to display directly in jupyter notebook replace the **draw_registrations** with this:

```python
from open3d import JVisualizer

def draw_registrations(source, target, transformation = None, recolor = False):
        source_temp = copy.deepcopy(source)
        target_temp = copy.deepcopy(target)
        if(recolor):
            source_temp.paint_uniform_color([1, 0.706, 0])
            target_temp.paint_uniform_color([0, 0.651, 0.929])
        if(transformation is not None):
            source_temp.transform(transformation)
        visualizer = JVisualizer()
        visualizer.add_geometry(source_temp)
        visualizer.add_geometry(target_temp)
        visualizer.show()
```

In [ ]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import copy
        
# Helper function to draw registrations (reccomended)
def draw_registrations(source, target, transformation = None, recolor = False):
        source_temp = copy.deepcopy(source)
        target_temp = copy.deepcopy(target)
        if(recolor):
            source_temp.paint_uniform_color([1, 0.706, 0])
            target_temp.paint_uniform_color([0, 0.651, 0.929])
        if(transformation is not None):
            source_temp.transform(transformation)
        o3d.visualization.draw_geometries([source_temp, target_temp])

# sum of squared differences
def ssd(source, target):
    return np.sum(np.square(source - target))

## Creating pointclouds from image data
Now we are going to try to create our own pointclouds from rgb and depth images.

In [ ]:
# Read in images. We have images 000000 - 0000400
color_raw0 = o3d.io.read_image("RGBD/color/000000.jpg")
depth_raw0 = o3d.io.read_image("RGBD/depth/000000.png")

color_raw1 = o3d.io.read_image("RGBD/color/000005.jpg")
depth_raw1 = o3d.io.read_image("RGBD/depth/000005.png")

Create pointclouds from rgb + depth images.

If you set *convert_rgb_to_intensity = False* you will retain the colors from the rgb image.

In [ ]:
rgbd_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw0, 
    depth_raw0, 
    convert_rgb_to_intensity = True)

rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw1, 
    depth_raw1, 
    convert_rgb_to_intensity = True)

#show images
fig= plt.figure(figsize=(15,15))
plt.subplot(221)
plt.title('Redwood grayscale0 image')
plt.imshow(rgbd_image0.color, cmap='gray')

plt.subplot(222)
plt.title('Redwood depth0 image')
plt.imshow(rgbd_image0.depth)

plt.subplot(223)
plt.title('Redwood grayscale1 image')
plt.imshow(rgbd_image1.color, cmap='gray')

plt.subplot(224)
plt.title('Redwood depth1 image')
plt.imshow(rgbd_image1.depth)

plt.show()


## Images to Pointcloud
Now we create point clouds from the rgbd images we just created.


Here we use **PinholeCameraIntrinsicParameters.PrimeSenseDefault** as default camera parameter. 

It has image resolution 640x480, focal length (fx, fy) = (525.0, 525.0), and optical center (cx, cy) = (319.5, 239.5). 

An identity matrix is used as the default extrinsic parameter. pcd.transform applies an up-down flip transformation on the point cloud for better visualization purpose.


If it becomes too slow you can downsample the pointcloud.

In [ ]:
# Source pointcloud
camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

source = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image0, camera)

# Target pointcloud
target = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image1, camera)

# Flip it, otherwise the pointcloud will be upside down
source.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
target.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Draw
draw_registrations(source, target, recolor=True)


### Evaluation of pointclouds

Before we can run ICP we evaluate our source and target pointclouds. This gives us a measure to see if we need a better initial transformation or not.

[The function evaluate_registration calculates two main metrics. fitness measures the overlapping area (# of inlier correspondences / # of points in target). Higher the better. inlier_rmse measures the RMSE of all inlier correspondences. Lower the better.](http://www.open3d.org/docs/latest/tutorial/Basic/icp_registration.html)

In [ ]:
# Parameters
threshold = 0.02
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

#Evaluate registration
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

## ICP

Now try to call icp with your point clouds and your initial transformation.

Initially we use:
```Python
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

icp_result = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    point_to_plane)

```

In [ ]:
###
# ICP code here
###

point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
##############
# Downsample #
##############
voxel_size = 0.05
# 1 Downsample both point clouds
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)
# 2 Estimate surface normals of pointclouds
source.estimate_normals()
target.estimate_normals()

icp_result = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    point_to_plane)
    
draw_registrations(source, target, icp_result.transformation, True)

# Exersices

### A)
If you increase the amount of steps from the original image so from i.e. 000000-000001 to 00000-000300 what happens?


In [ ]:
# Read in images. We have images 000000 - 0000400
color_raw0 = o3d.io.read_image("RGBD/color/000001.jpg")
depth_raw0 = o3d.io.read_image("RGBD/depth/000001.png")

color_raw1 = o3d.io.read_image("RGBD/color/000301.jpg")
depth_raw1 = o3d.io.read_image("RGBD/depth/000301.png")

#####################
# Recombine to RGBD #
#####################
rgbd_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw0, 
    depth_raw0, 
    convert_rgb_to_intensity = True)

rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw1, 
    depth_raw1, 
    convert_rgb_to_intensity = True)

# Create camera model
camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

# Source pointcloud
source = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image0, camera)

# Target pointcloud
target = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image1, camera)

# Flip it, otherwise the pointcloud will be upside down
source.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
target.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

##############
# Downsample #
##############
voxel_size = 0.01
# 1 Downsample both point clouds
source.voxel_down_sample(voxel_size=voxel_size)
target.voxel_down_sample(voxel_size=voxel_size)
# 2 Estimate surface normals of pointclouds
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)

###########
# Fitness #
###########

# Parameters
threshold = 0.02
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

#Evaluate registration
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

#######
# ICP #
#######

point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

icp_result = o3d.pipelines.registration.registration_icp(
    source,
    target,
    max_correspondence_distance=threshold,
    init=trans_init, 
    estimation_method=point_to_plane
    )

In [ ]:
draw_registrations(source, target, icp_result.transformation, recolor=True)



### B)
Can you tweak the parameters *threshold* and *trans_init* to combat some of the ill effects that starts appearing?


In [ ]:
###########
# Fitness #
###########

# Parameters
threshold = 1.5 # Increased so we can detect the correct points
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

#Evaluate registration
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

#######
# ICP #
#######

point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

icp_result = o3d.pipelines.registration.registration_icp(
    source,
    target,
    max_correspondence_distance=threshold,
    init=trans_init, 
    estimation_method=point_to_point
    )

In [ ]:
draw_registrations(source, target, icp_result.transformation, recolor=True)


### C)
Again try to use 
```Python
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    point_to_plane)
```

This requires you to find the normals for each point cloud use:
```python
    source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                            max_nn=30),fast_normal_computation=True)
```
Compare the resulting translations of the two methods is one better than the other?


In [ ]:
# Parameters
threshold = 1.5 # Increased so we can detect the correct points
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    point_to_plane)

In [ ]:
draw_registrations(source, target, reg_p2p.transformation, recolor=True)


### D)
Extend this and try to see how much of the bedroom you can reconstruct from the rgb and depth images.
you can extend a pointcloud by new = source + target remember to resample the point cloud some times so it does not get too large down_source = source.voxel_down_sample(voxel_size=0.05)

In [ ]:
def add_new_pc(source, no, voxel_size=0.005):
    # Read in images. We have images 000000 - 0000400
    if no<10:
        no = "00000"+str(no)
        color_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
        depth_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
    elif 10<=no<100:
        no = "0000"+str(no)
        color_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
        depth_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
    elif 100<=no<1000:
        no = "000"+str(no)
        color_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
        depth_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
    else:
        return source

    #####################
    # Recombine to RGBD #
    #####################
    rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw1, 
        depth_raw1, 
        convert_rgb_to_intensity = True)

    # Create camera model
    camera = o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

    # Target pointcloud
    target = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image1, camera)

    # Flip it, otherwise the pointcloud will be upside down
    target.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

    ##############
    # Downsample #
    ##############
    # 1 Downsample both point clouds
    #source = source.voxel_down_sample(voxel_size=voxel_size)
    #target = target.voxel_down_sample(voxel_size=voxel_size)
    # 2 Estimate surface normals of pointclouds
    source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                            max_nn=30),fast_normal_computation=True)
    target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                            max_nn=30),fast_normal_computation=True)

    #######
    # ICP #
    #######
    # Parameters
    threshold = 0.02
    trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

    #point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
    point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

    icp_result = o3d.pipelines.registration.registration_icp(
        source,
        target,
        max_correspondence_distance=threshold,
        init=trans_init, 
        estimation_method=point_to_plane
        )
    print(no, end="... ")
    # Transform the source points to targets coordinates, and add target
    new = source.transform(icp_result.transformation) + target #.voxel_down_sample(voxel_size=voxel_size)
    return new

In [ ]:
def whole_process(source, voxel_size=0.005):
    
    for i in range(1, 30):
        # Read in images. We have images 000000 - 0000400
        no = 0
        if i<10:
            no = "00000"+str(i)
            color_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
            depth_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
        elif 10<=i<100:
            no = "0000"+str(i)
            color_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
            depth_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
        elif 100<=i<1000:
            no = "000"+str(i)
            color_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
            depth_raw1 = o3d.io.read_image("RGBD/color/"+no+".jpg")
        else:
            return source
        print(no, end="... ")

        #####################
        # Recombine to RGBD #
        #####################
        rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
            color_raw1, 
            depth_raw1, 
            convert_rgb_to_intensity = True)

        # Create camera model
        camera = o3d.camera.PinholeCameraIntrinsic(
                o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

        # Target pointcloud
        target = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image1, camera)

        # Flip it, otherwise the pointcloud will be upside down
        target.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

        ##############
        # Downsample #
        ##############
        # 1 Downsample both point clouds
        #source = source.voxel_down_sample(voxel_size=voxel_size)
        #target = target.voxel_down_sample(voxel_size=voxel_size)
        # 2 Estimate surface normals of pointclouds
        source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                                max_nn=30),fast_normal_computation=True)
        target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                                max_nn=30),fast_normal_computation=True)

        #######
        # ICP #
        #######
        # Parameters
        threshold = 0.02
        trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

        #point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
        point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

        icp_result = o3d.pipelines.registration.registration_icp(
            source,
            target,
            max_correspondence_distance=threshold,
            init=trans_init, 
            estimation_method=point_to_plane
            )
        
        # Transform the source points to targets coordinates, and add target
        source += target.transform(icp_result.transformation) #.voxel_down_sample(voxel_size=voxel_size)
    return source

In [ ]:
_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw0, 
    depth_raw0, 
    convert_rgb_to_intensity = True)
    
# Create camera model
camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

# Source pointcloud
source = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image0, camera)

# Flip it, otherwise the pointcloud will be upside down
source.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# 1 Downsample both point clouds
source = source.voxel_down_sample(voxel_size=voxel_size)
# 2 Estimate surface normals of pointclouds
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                            max_nn=30),fast_normal_computation=True)

################
# Do the stuff #
################
#for i in range(1, 100):
    #source = add_new_pc(copy.deepcopy(source), i, voxel_size=voxel_size)
    #if i % 10 == 0:
    #    source=source.voxel_down_sample(voxel_size=voxel_size)

result = whole_process(copy.deepcopy(source), voxel_size=voxel_size)

In [ ]:
result

In [ ]:
o3d.visualization.draw_geometries([result])

In [ ]:
from pyoints import (
    storage,
    Extent,
    transformation,
    filters,
    registration,
    normals,
)

In [ ]:
class Map():
    def __init__(self):
        self.map = np.array([]) # Pointcloud for map [[x, y], [x, y], ...]

    def add_point(self, x, y):
        self.map = np.append(self.map, [[x, y]], axis=0) # Add new measurement to map
    
    def add_pointcloud(self, pointcloud):
        '''Takes a pointcloud and run icp to align it with the map and add it to the map
        input:
        pointcloud: [[x, y], [x, y], ...]
        '''
        pc = self.run_icp(pointcloud)
        self.map = np.append(self.map, pc, axis=0)

    def run_icp(self, pointcloud, max_iter, min_delta_err, init_T=np.eye(3)):
        '''Run icp to align a pointcloud with the map
        input:
        pointcloud: [[x, y], [x, y], ...]
        '''
        # downsample pointcloud
        # pointcloud = sklearn.utils.resample(pointcloud, n_samples=100, replace=False, random_state=0)
        print(pointcloud.shape)
        pointcloud = np.random.choice(pointcloud.shape[0], 100, replace=False)
        print(pointcloud.shape)
        
        point_dict = {
            'A': self.map,
            'B': pointcloud
        }

        d_th = 0.04
        radii = [d_th, d_th, d_th]
        icp = registration.ICP(
            radii,
            max_iter=60,
            max_change_ratio=0.000001,
            k=1
        )

        T_dict, pairs_dict, report = icp(point_dict)
        T = T_dict['B']
        return T @ pointcloud.T

In [ ]:
 import cv2
 import numpy as np
 import matplotlib.pyplot as plt
 from sklearn.neighbors import NearestNeighbors

In [ ]:
def icp(a, b, init_pose=(0,0,0), no_iterations = 100):
    '''
    The Iterative Closest Point estimator.
    Takes two cloudpoints a[x,y], b[x,y], an initial estimation of
    their relative pose and the number of iterations
    Returns the affine transform that transforms
    the cloudpoint a to the cloudpoint b.
    Note:
        (1) This method works for cloudpoints with minor
        transformations. Thus, the result depents greatly on
        the initial pose estimation.
        (2) A large number of iterations does not necessarily
        ensure convergence. Contrarily, most of the time it
        produces worse results.
    '''

    src = np.array([a.T], copy=True).astype(np.float32)
    dst = np.array([b.T], copy=True).astype(np.float32)

    #Initialise with the initial pose estimation
    Tr = np.array([[np.cos(init_pose[2]),-np.sin(init_pose[2]),init_pose[0]],
                   [np.sin(init_pose[2]), np.cos(init_pose[2]),init_pose[1]],
                   [0,                    0,                   1          ]])

    src = cv2.transform(src, Tr[0:2])

    for i in range(no_iterations):
        #Find the nearest neighbours between the current source and the
        #destination cloudpoint
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(dst[0])
        distances, indices = nbrs.kneighbors(src[0])

        #Compute the transformation between the current source
        #and destination cloudpoint
        T, inliers = cv2.estimateAffine2D(src, dst[0, indices.T]) # retval, inliers
        #Transform the previous source and update the
        #current source cloudpoint
        src = cv2.transform(src, T)
        #Save the transformation from the actual source cloudpoint
        #to the destination
        Tr = np.dot(Tr, np.vstack((T,[0,0,1])))
    return Tr[0:2]

In [ ]:
#Create the datasets
ang = np.linspace(-np.pi/2, np.pi/2, 50)
a = np.array([ang, np.sin(ang)])
th = np.pi/2
rot = np.array([[np.cos(th), -np.sin(th)],[np.sin(th), np.cos(th)]])
b = np.dot(rot, a) + np.array([[0.2], [0.3]])


#Run the icp
M2 = icp(b, a, [0.1,  0.33, np.pi/2.2], 100)

#Plot the result
src = np.array([a.T]).astype(np.float32)
res = cv2.transform(src, M2)
plt.figure()
plt.plot(a[0], a[1], 'b.')
plt.plot(b[0],b[1], 'r.')
plt.plot(res[0].T[0], res[0].T[1], 'g.')
plt.show()

In [3]:
from matplotlib import pyplot as plt
from simpleicp import PointCloud, SimpleICP
import numpy as np

# Read point clouds from xyz files into n-by-3 numpy arrays
X_fix = np.genfromtxt("bunny_part1.xyz")
X_mov = np.genfromtxt("bunny_part2.xyz")


# X_fix = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 1]])
# X_mov = X_fix + 0.1


# print(X_fix.shape)


# Create point cloud objects
pc_fix = PointCloud(X_fix, columns=["x", "y", "z"])
pc_mov = PointCloud(X_mov, columns=["x", "y", "z"])
# plt.plot(X_fix[:, 0], X_fix[:, 1], 'b.')
# plt.plot(X_mov[:, 0], X_mov[:, 1], 'r.')
# plt.show()

print(pc_fix)
# Create simpleICP object, add point clouds, and run algorithm!
icp = SimpleICP()
icp.add_point_clouds(pc_fix, pc_mov)
H, X_mov_transformed, rigid_body_transformation_params = icp.run(max_overlap_distance=1)



          x     y      z  selected
0     -3.73 -0.78  12.79      True
1     -4.44 -0.58  12.89      True
2     -6.45 -4.30  15.12      True
3     -0.03 -2.33  13.02      True
4     -2.19 -0.91  12.67      True
...     ...   ...    ...       ...
20697 -4.45 -0.65  14.61      True
20698 -5.19 -0.46  14.29      True
20699 -4.84 -0.68  14.42      True
20700 -5.21 -0.47  14.14      True
20701 -5.07 -0.48  14.42      True

[20702 rows x 4 columns]
Consider partial overlap of point clouds ...
Select points for correspondences in fixed point cloud ...
Estimate normals of selected points ...
Start iterations ...
iteration | correspondences | mean(residuals) |  std(residuals)
   orig:0 |             863 |          0.0403 |          0.1825
        1 |             862 |          0.0096 |          0.1113
        2 |             775 |          0.0050 |          0.0553
        3 |             807 |          0.0022 |          0.0407
        4 |             825 |          0.0016 |          0.0346
     